# Decision Trees

Decision Trees, like SVMs, are versatile ML algos that can do both classification and regression (and even multioutput tasks!). They're quite powerful, capable of fitting complex datasets. They're also the fundamental component of Random Forests (see next chapter).

## Training and Visualizing a Decision Tree

Let's start by building one and trying it out on the iris dataset to see how it makes predictions.

In [1]:
%matplotlib inline
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import numpy as np

iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

## Making Predictions

The way that Decision Trees work is by separating out the data into a series of questions that determine which way to go down the tree. It starts at the root node and works its way down to a leaf node, at which a decision is made. __One important quality of decision trees is that they do not require feature scaling *or* centering at all__.

A node's `samples` attribute counts how many training instances it applies to. A node's `value` attribute tells you how many training instances of each class this node applies to. Lastly, a node's `gini` attribute measures its *impurity*; a node is "pure" (`gini = 0`) if all training instances it applies to beling to the same class. The equation for the `gini` score is given below:

$$ G_i = 1 - \sum_{k=1}^n(p_{i,k})^2$$

* $p_{i,k}$ is the ratio of class $k$ instances to the training instances in the $i_{th}$ node

*Note: sklearn uses the CART algo, which produces only __binary trees__. Other algos such as ID3 can produce Decision Trees with nodes that have more than two children.*

The `max_depth` hyperparameter sets the number of levels deep the decision tree will go to create the tree.

#### Quick Sidenote about Model Interpretation

Generally speaking, there are two kinds of ML models. The first kind, *white box models* are like this decision tree; they are fairly intuitive and their decisions are easy to interpret and follow. On the opposite side, *black box models* (like neural networks and random forests) are ML models that make great predictions and can have their calcuations easily checked, but an explanation for why the model chose what it chose is hard to make. For example, if a neural net identifies a person in a photo, it's hard to know what exactly the model used to decide this. Conversely, Decision Trees provide nice and simple classification rules that can even be applied manually if need be!

## Estimating Class Probabilities

A Decision Tree can also estimate the probability that an instance belongs to a particular class $k$: first, it traverses the tree to find the leaf node for this instance, and then it returns the ratio of training instances of class $k$ in this node.

As an example, suppose you had a flower with petals 5 cm long and 1.5 cm wide. This should fall as a depth-2 leaf node (in our classifier we trained), so the classifier should output 0% for Iris-Setso, 90.7% for Iris-Versicolor, and 9.3% for Iris-Virginica. If you had it predict the class, it would (obviously) output Iris-Versicolor (class #1). Let's try it out

In [2]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [3]:
tree_clf.predict([[5, 1.5]])

array([1])

## The CART Training Algorithm

Sklearn uses the *Classification And Regression Tree* (CART) algorithm to train decision trees (a.k.a. "growing" trees). The idea is pretty simple: the algorithm first splits the training set into two subsets using a single feature $k$ and a threshold $t_k$ (something like petal length ≤ 2.45 cm). How does it choose $k$ and $t_k$? It searches for the pair $(k, t_k)$ that produces the purest subsets (weighted by size). The corresponding cost function that the algo tries to minimize is given below:

$$J(k, t_k) = \frac{m_{\text{left}}}{m}G_{\text{left}} + \frac{m_{\text{right}}}{m}G_{\text{right}} \\
\text{ where }\left\{ \begin{array}{ll}G_{\text{left/right}} \text{ measures the impurity of the left/right subset}, \\
m_{\text{left/right}} \text{ is the number of instances in the left/right subset}\end{array}\right.$$

Once it has successfully split the training set in two, it splits the subsets using the same logic, then the sub-subset, then the... you get the point. It does so recursively, stopping once it reaches the maximum depth or if it cannot find a split that will reduce impurity. A few other hyperparameters contain additional stopping conditions, like `min_samples_split`, `min_samples_leaf`, `min_weight_fraction_leaf`, and `max_leaf_nodes`. This algorithm is a __*greedy*__ algorithm, meaning that it tries to make the most possible progress towards the end goal with each step. This means that the algorithm doesn't check whether or not the split will lead to the eventual lowest possible impurity, but rather just goes for it right away. These kinds of algos will often produce a pretty good solution, but it's not guaranteed to be anywhere near optimal.

Unfortunately, finding the optimal tree is known to be an *NP-Complete* problem. It requires $O(e^m)$ time to complete, making the problem intractable (even for fairly small training sets). Because of this, "good enough" will have to suffice.

### Computational Complexity

Making predictions requires traversing the full tree from root to a leaf. Decision Trees are (generally) approximately balanced, so traversing the tree will take roughly $O(log_2(m))$ nodes. This is pretty fast, since each comparison is only checking the value of one feature per node.

The training algorithm, however, has to compare all features (or less if the `max_features` hyperparameter is set) on all samples at each node. This results in a training complexity of $O(\text{n x m }log(m))$. For small training sets (less than a few thousand instances), sklearn can speed up training by presorting the data (`presort=True`). but this slows down training considerably for larger training sets.

### Gini Impurity or Entropy

The Gini impurity measure is used by default, but the *entropy* measure can be selected instead by changing the value of the `criterion` hyperparameter to "`entropy`". The term is taken from the meaning of the term in thermodynamics (which relates to disorder of molecules). The more still and well-ordered the molecules are, the more entropy approaches 0. In ML, entropy is frequently used as an impurity measure; a set's entropy is 0 when it contains instances of only one class. For the $i^{th}$ node, entropy is defined as follows:

$$H_i = - \mathop{\sum_{k=1}^n}_{p_{i, k} \neq 0}p_{i, k}\log(p_{i, k})$$

So which measure to use? Most of the time, it won't matter at all (they lead to similar trees). Gini is slightly faster to compute, so it's a good default. However, when they do differ, Gini tends to isolate the most frequent class in its own branch whilst entropy tends to produce slightly more balanced trees.

### Regularization Hyperparameters

Decision Trees don't make many assumptions about the training data (as opposed to linear models which obviously assume that the data is linear, for example). If left unconstrained, the tree will adapt to the training data very closely and will most likely overfit it. Such a model is often called a *nonparametric model*, not because it doesn't have any parameters, but because the number of parameters isn't determined prior to training, so the model structure is free to stick closely to the data. In contrast, a *parametric model* such as a linear model has a predetermined number of parameters, so its degree of freedom is limited, reducing the risk of overfitting (but increasing the risk of underfitting).

To avoid overfitting, you must restrict the tree's freedom during training. This process is called regularization (as you know by now). Regularization hyperparams depend on the algo used, but generally, you can at least restrict the max depth of the tree. In sklearn, this is controlled by the `max_depth` hyperparam (the default value is `None` (meaning unlimited). Reducing `max_depth` reduces the risk of overfitting by regularization of the model.

The `DecisionTreeClassifier` has a few other params that similarly restrict the shape of the tree:

* `min_samples_split` is the minimum number of samples a node must have before it can be split
* `min_samples_leaf` is the minimum number of samples a leaf node must have
* `min_weight_fraction_leaf` is the same as `min_samples_leaf` but expressed as a fraction of the total number of weighted instances
* `max_leaf_nodes` is the max number of leaf nodes
* `max_features` maximum number of features that are evaluated for splitting at each node. Increaing the `min_*` or decreaing the `max_*` hyperparameters will regularize the model.

Other algos work by first training the Tree without restrictions, then it will *prune* (delete) unnecessary nodes. A node with all leaf nodes being children is considered unnecessary if the purity improvement it provides isn't *statistically significiant*. Standard stats test like $\chi^2$ are used to estimate the probability that the improvement is purely the result of chance (the null hypothesis). If this p-value is higher than the threshold decided (usually 0.05), the node is considered useless and its children are deleted. This continues until all unnecessary nodes are removed

### Regression

Decision Trees can also do regression! We'll use sklearn's `DecisionTreeRegressor` class to do so (with `max_depth=2`):

In [4]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

*See page 177 for the resulting tree*

The resulting tree looks like the classification tree from earlier, but instead of predicting a class at each node, it predicts a singular value. The final prediction for that node ends up being the average target value of the 110 training instances associated to that particular node.

The CART algo works pretty much the same as before, but instead of trying to split the training set in a way that minimizes impurity, it now tries to split that training set in a way that minimizes the MSE. The cost function for this is given below:

$$J(k, t_k) = \frac{m_{\text{left}}}{m}MSE_{\text{left}} + \frac{m_{\text{right}}}{m}MSE_{\text{right}} \\
\text{ where }\left\{ \begin{array}{ll}MSE_{\text{node}} = \sum_{i \in \text{node}} (\hat{y}_{\text{node}} - y^i)^2 \\
\hat{y}_{\text{node}} = \frac{1}{m_{\text{node}}}\sum_{i \in \text{node}}y^i\end{array}\right.$$

This model is also prone to overfitting as well. Without any regularization (a.k.a. using the default hyperparameters), you get predictions that will badly overfit the data.

### Instability

For as good as Decision Trees are, they have plenty of limitations. Firstly, they love orthogonal decision boundaries which makes them sensitive to training set rotation. Rotating the data may make your decision tree not generalize well. One way to limit this problem is to use PCA (see chapter 8) which will often result in a better orientation of the training data.

More generally, the main issue with Decision Trees is that they are very sensitive to small variations in the training data. For example, removing just the widest Iris-Versicolor from the iris training set and train a new Decision Tree, you may get a totally different model.

*Note: since the training algo used in sklearn is stochastic, you may get very different models even on the same training data (unless you set the `random_state` hyperparameter*

Random Forests can limit this instability by averaging predictions over many trees (see next chapter).

# 7) Train and Fine-Tune a Decision Tree for the moons dataset

In [5]:
# Train and fine-tune a Decision Tree for the moons dataset
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Use grid search with cross-validation (using GridSearchCV) to find good hyperparameters
from sklearn.model_selection import GridSearchCV

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1)

grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:    2.8s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 'min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None

In [7]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=17,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [8]:
# Train it on training set
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

# 8) Grow a Forest

In [10]:
# 1000 subsets of 100 randomly selected instances

from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = list()

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [12]:
# Train one decision tree on each subset, using the best hyperparameter values found ealier. Evaluate 1,000 decision
# trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than
# the first tree (about 80% accuracy)
from sklearn.base import clone

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = list()

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    
np.mean(accuracy_scores)

0.8054494999999999

In [13]:
# Now for the magic. For each test set instance, generate the predictions of the 1,000 decision trees and keep
# only the most frequent predictions (you can use SciPy's mode() function for this) This gives you majority-vote
# predictions over the test sets
from scipy.stats import mode

Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)
    
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872